In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import sqlite3, time
from bs4 import BeautifulSoup

# --- DB 연결 ---
conn = sqlite3.connect("kbo.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS game_schedule (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  game_date TEXT NOT NULL,
  weekday TEXT,
  game_time TEXT,
  team1 TEXT,
  team2 TEXT,
  score TEXT,
  gamecenter_url TEXT,
  stadium TEXT
);
""")
conn.commit()

# --- Selenium 설정 ---
chrome_opts = Options()
chrome_opts.add_argument("--headless=new")  # 브라우저 창 숨김
chrome_opts.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_opts)

driver.get("https://www.koreabaseball.com/schedule/schedule.aspx")
time.sleep(3)  # 페이지 로딩 대기

# --- HTML 가져오기 ---
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# --- 테이블 찾기 ---
table = soup.find("table", {"class": "tbl"})
if not table:
    print("⚠️ 경기 일정 테이블을 찾을 수 없습니다.")
    exit()

rows = table.find_all("tr")

# --- 데이터 추출 ---
for row in rows:
    cols = row.find_all("td")
    if len(cols) < 7:
        continue

    date_text = cols[0].get_text(strip=True)
    if not date_text or "(" not in date_text:
        continue

    game_date, weekday = date_text.split("(")
    weekday = weekday.replace(")", "")
    game_time = cols[1].get_text(strip=True)
    game_text = cols[2].get_text(strip=True)

    # 경기 텍스트에서 팀/점수 분리
    team1, team2, score = None, None, None
    try:
        if "vs" in game_text:
            parts = game_text.split("vs")
            left = parts[0].strip()
            right = parts[1].strip()

            if " " in left:
                team1, left_score = left.split(" ", 1)
            else:
                team1, left_score = left, ""

            if " " in right:
                right_score, team2 = right.split(" ", 1)
            else:
                right_score, team2 = "", right

            score = f"{left_score.strip()} vs {right_score.strip()}" if (left_score or right_score) else ""
    except:
        continue

    review_link = ""
    btn = cols[3].find("a")
    if btn and "href" in btn.attrs:
        review_link = "https://www.koreabaseball.com" + btn["href"]

    stadium = cols[-1].get_text(strip=True)

    cur.execute("""
        INSERT INTO game_schedule
        (game_date, weekday, game_time, team1, team2, score, gamecenter_url, stadium)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (game_date, weekday, game_time, team1, team2, score, review_link, stadium))

conn.commit()
conn.close()
print("✅ Selenium으로 KBO 경기 일정 수집 완료!")


✅ Selenium으로 KBO 경기 일정 수집 완료!
